In [2]:
import pandas as pd
import numpy as np
import math
import random
import scipy
import statsmodels.formula.api as smf
import sklearn.linear_model as lm

import keras

import matplotlib.pyplot as plt
%matplotlib inline

# Data analysis

In [39]:
#Global Variables 
train_test_split = .8 #fraction of data to use for training

# Model Fitting

In [82]:
pdth = pd.read_csv("ph_v1_days.csv")
# print(pdth.head())
print("Total Availible Keys: ", len(pdth.keys()))

repl_nan = lambda x: list(map(lambda xx: 0 if math.isnan(xx) else xx, x))

AVG_ph = list(map(lambda x: 1 if x<6 else 0, repl_nan(pdth['PH_evening'])))

xs = []
for k in pdth.keys():
    if k in ["Date", "PH_morning", "PH_midday", "PH_evening", "PH_avg"]:
        continue
    xs.append(repl_nan(pdth[k]))
    
# Put the data into numpy arrays
xs=np.transpose(np.array(xs))
ys=np.array(AVG_ph)

#Xshape = (num_samples, features), Yshape = (num_samples,)
print("\nPre Split Shapes:")
print("X shape = {}, Y shape = {}".format(xs.shape, ys.shape))

#Create a training and testing split

#Zip the xs and ys together
xsys = list(zip(xs, ys))

#shuffle the combined list
random.shuffle(xsys)

#Unzip the shuffled list
xs, ys = zip(*xsys)

splt_ndx = int(len(xs)*train_test_split)

xs_tr = np.array(xs[0:splt_ndx]); xs_ts = np.array(xs[splt_ndx:])
ys_tr = np.array(ys[0:splt_ndx]); ys_ts = np.array(ys[splt_ndx:])

#Print the final shapes
print("\nFinal Shapes:")
print("Shapes: X train = {}, Y train = {}".format(xs_tr.shape, ys_tr.shape))
print("        X test  = {},  Y test  = {}".format(xs_ts.shape, ys_ts.shape))

print(xs_tr[0])

Total Availible Keys:  99

Pre Split Shapes:
X shape = (212, 94), Y shape = (212,)

Final Shapes:
Shapes: X train = (169, 94), Y train = (169,)
        X test  = (43, 94),  Y test  = (43,)
[0.00e+00 1.00e+00 1.85e+03 2.50e+02 0.00e+00 6.00e+02 3.00e+02 0.00e+00
 5.00e+02 0.00e+00 0.00e+00 0.00e+00 2.00e+02 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.60e+02 0.00e+00
 0.00e+00 1.50e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.40e+02 0.00e+00 0.00e+00
 7.00e+01 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 5.00e+01
 0.00e+00 0.00e+00 0.00e+00 1.50e+02 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+00 5.00e+01 0.00e+00 0.00e+00 0.00e+00 1.00e+02 0.00e+00 0.00e+00
 0.00e+00 5.00e+01 4.50e+01 6.00e+01 0.00e+00 0.00e+00 0.00e+00 0.00e+00
 0.00e+0

In [75]:
model_lr = lm.LogisticRegression(solver='liblinear')
model_lr.fit(xs_tr, ys_tr)

get_acc_lr = lambda x, y: len(np.where(model_lr.predict(x)==y)[0])/len(y)

print("Training Accuracy = ", get_acc_lr(xs_tr, ys_tr))
print("Testing  Accuracy = ", get_acc_lr(xs_ts, ys_ts))

print(model_lr.predict(xs_tr)[0:5])
print(ys_tr[0:5])


Training Accuracy =  1.0
Testing  Accuracy =  0.46511627906976744
[0 1 1 1 1]
[0 1 1 1 1]


In [80]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

hddn_units = 64
model_ml = Sequential([
    Dense(hddn_units, activation='relu'), 
    Dropout(.5),
    Dense(1, activation='sigmoid')
])
model_ml.compile(optimizer='adam', loss='mse')

model_ml.fit(xs_tr, ys_tr, epochs=100, verbose=0)

In [81]:
get_acc_ml = lambda x, y: len(np.where(model_ml.predict(x)==y)[0])/len(y)

print("Training Accuracy = ", get_acc_ml(xs_tr, ys_tr))
print("Testing  Accuracy = ", get_acc_ml(xs_ts, ys_ts))

Training Accuracy =  99.0
Testing  Accuracy =  27.0
